In [5]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import misc
import imageio
import os
import pandas as pd
import mahotas as mh

In [6]:
# folder path
dir_path = r'BestSegmentations/'
# list to store files
res = [] # files name
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        res.append(path)

In [4]:
gmic_iou_otsu = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'], index=range(370))
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  gmic_otsu = cv2.imread(('GrayHeatmapsGmic/' + res1.split(".")[0] + '_CAM1.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(gmic_otsu)
  otsu_threshold, image_otsu = cv2.threshold(gmic_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) 
  intersection = np.logical_and(image_otsu, segm1)
  union = np.logical_or(image_otsu, segm1)
  iou = np.sum(intersection) / np.sum(union)
  gmic_iou_otsu.loc[i] = (res1.split(".")[0], otsu_threshold, iou)
  # Save image with contours--------------------------------------------------------------------------------------------
  #orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  #contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #contours_otsu, hierarchy1 = cv2.findContours(image_otsu, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  #cv2.drawContours(orig, contours_otsu, -1, (255,255,0), 6)
  #cv2.imwrite(('ImageOtsu/Gmic/' + res1.split(".")[0] + '_CAM1OT.png'), orig)

In [5]:
np.sum(gmic_iou_otsu['IoU'])/370
# antes 0.07458446169374482

0.07867563597374283

In [6]:
gmic_iou_otsu

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,21,0.0148981
1,173ca_256_R_MLO,25,0
2,168ca_248_L_MLO,5,0
3,137ca_208_R_CC,5,0
4,133ca_203_L_MLO,0,0
...,...,...,...
365,14ca_19_R_MLO,2,0.0932385
366,140ca_211_L_MLO,8,0.000505537
367,23ca_38_L_CC,4,0.0994693
368,13ca_18_R_CC,36,0


In [7]:
gmic_iou = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
for i in range(0, 370, 1):
  segm1 = imageio.imread(('BestSegmentations/' + res[i]))
  res1 = str(res[i])
  gmic = cv2.imread(('GrayHeatmapsGmic/' + res1.split(".")[0] + '_CAM1.png'), cv2.IMREAD_UNCHANGED)
  assert np.shape(segm1) == np.shape(gmic)
  iou_score = []
  for n in range(0, 255, 1):
    th , image = cv2.threshold(gmic, n, 255, cv2.THRESH_BINARY) 
    intersection = np.logical_and(image, segm1)
    union = np.logical_or(image, segm1)
    iou = np.sum(intersection) / np.sum(union)
    iou_score.append(iou)
  threshold = iou_score.index(np.max(iou_score))
  IoU = np.max(iou_score)
  gmic_iou.loc[i] = (res1.split(".")[0], threshold, IoU)
  # Save image with contours--------------------------------------------------------------------------------------------
  #orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'),0)
  #th , image = cv2.threshold(gmic, threshold, 255, cv2.THRESH_BINARY)
  #contours_seg, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #contours_sweep, hierarchy1 = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  #cv2.drawContours(orig, contours_seg, -1, (0,255,0), 6)
  #cv2.drawContours(orig, contours_sweep, -1, (255,255,0), 6)
  #cv2.imwrite(('ImageSweep/Gmic/' + res1.split(".")[0] + '_CAM1SW.png'), orig)

In [8]:
np.sum(gmic_iou['IoU'])/370

0.11023776020675827

In [4]:
gmic_iou_otsu_ = pd.DataFrame(columns=['image_index', 'threshold_otsu', 'IoU'])
gmic_iou_no_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
otsu_ = 0
no_ = 0
for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    gmic_otsu = cv2.imread(('GrayHeatmapsGmic/' + res1.split(".")[0] + '_CAM1.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(gmic_otsu)
    otsu_threshold, image_otsu = cv2.threshold(gmic_otsu, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inter_out = np.sum(np.logical_and(image_otsu, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_otsu)
        new_image = np.zeros(np.shape(image_otsu))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        gmic_iou_otsu_.loc[otsu_] = (res1.split(".")[0], otsu_threshold, iou)
        otsu_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageOtsu/Gmic/' + res1.split(".")[0] + '_CAM1OT.png'), orig)
    else:
        gmic_iou_no_.loc[no_] = (res1.split(".")[0], otsu_threshold)
        no_ += 1

In [5]:
gmic_iou_otsu_

,image_index,threshold_otsu,IoU
0,123ca_187_L_CC,21.0,0.016281
1,151ca_225_L_MLO,33.0,0.266914
2,179ca_264_L_MLO,3.0,0.099130
3,28ca_45_R_CC,35.0,0.117304
4,162ca_241_L_MLO,2.0,0.012871
...,...,...,...
225,154ca_228_L_CC,4.0,0.184729
226,14ca_19_R_MLO,2.0,0.261980
227,140ca_211_L_MLO,8.0,0.000746
228,23ca_38_L_CC,4.0,0.221445


In [6]:
print('IoU Gmic Otsu = ', np.sum(gmic_iou_otsu_['IoU'])/otsu_)

IoU Gmic Otsu =  0.19776254632471704


In [7]:
gmic_iou_sw_ = pd.DataFrame(columns=['image_index', 'threshold', 'IoU'])
gmic_iou_n_ = pd.DataFrame(columns=['image_index', 'threshold_otsu'])
sw_ = 0
n_ = 0

for i in range(0, 370, 1):
    segm1 = imageio.imread(('BestSegmentations/' + res[i]))
    res1 = str(res[i])
    orig = cv2.imread(('original/' + res1.split(".")[0] + '.png'))
    gmic_sw = cv2.imread(('GrayHeatmapsGmic/' + res1.split(".")[0] + '_CAM1.png'), cv2.IMREAD_UNCHANGED)
    assert np.shape(segm1) == np.shape(gmic_sw)
    iou_score = []
    for n in range(0, 255, 1):
        th , image = cv2.threshold(gmic_sw, n, 255, cv2.THRESH_BINARY)      
        intersection = np.logical_and(image, segm1)
        union = np.logical_or(image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        iou_score.append(iou)
    threshold = iou_score.index(np.max(iou_score))
    IoU = np.max(iou_score)
    th , image_sw = cv2.threshold(gmic_sw, threshold, 255, cv2.THRESH_BINARY) 
    inter_out = np.sum(np.logical_and(image_sw, segm1))
    if inter_out != 0: 
        labelled, nr_objects = mh.label(image_sw)
        new_image = np.zeros(np.shape(image_sw))
        for i in range(1, nr_objects + 1, 1):
            mask = (labelled == i)
            inter = np.sum(np.logical_and(mask, segm1))
            if inter != 0:
                new_image = np.logical_or(new_image, mask)
        new_image = np.uint8(255*new_image)
        new_image = imageio.core.util.Array(new_image)
        intersection = np.logical_and(new_image, segm1)
        union = np.logical_or(new_image, segm1)
        iou = np.sum(intersection) / np.sum(union)
        gmic_iou_sw_.loc[sw_] = (res1.split(".")[0], th, iou)
        sw_ += 1
        contours, hierarchy = cv2.findContours(segm1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours1, hierarchy1 = cv2.findContours(new_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(orig, contours, -1, (255,255,0), 10)
        cv2.drawContours(orig, contours1, -1, (0,255,0), 10)
        cv2.imwrite(('ImageSweep/Gmic/' + res1.split(".")[0] + '_CAM1SW.png'), orig)
    else:
        gmic_iou_n_.loc[n_] = (res1.split(".")[0], otsu_threshold)
        n_ += 1

In [8]:
gmic_iou_sw_

,image_index,threshold,IoU
0,123ca_187_L_CC,1.0,0.237532
1,173ca_256_R_MLO,3.0,0.387577
2,168ca_248_L_MLO,2.0,0.540810
3,151ca_225_L_MLO,5.0,0.371875
4,179ca_264_L_MLO,1.0,0.174027
...,...,...,...
282,154ca_228_L_CC,0.0,0.064440
283,14ca_19_R_MLO,4.0,0.218042
284,140ca_211_L_MLO,0.0,0.103380
285,23ca_38_L_CC,6.0,0.188538


In [9]:
print('IoU Gmic Sweep = ', np.sum(gmic_iou_sw_['IoU'])/sw_)

IoU Gmic Sweep =  0.21099407622162253


In [4]:
# guardar una imagen para el paper 
segm1 = imageio.imread(('BestSegmentations/15ca_20_R_CC.bmp'))
orig = cv2.imread(('original/15ca_20_R_CC.png'))
gmic_sw = cv2.imread(('GrayHeatmapsGmic/15ca_20_R_CC_CAM1.png'), cv2.IMREAD_UNCHANGED)
assert np.shape(segm1) == np.shape(gmic_sw)
iou_score = []
for n in range(0, 255, 1):
    th , image = cv2.threshold(gmic_sw, n, 255, cv2.THRESH_BINARY)      
    intersection = np.logical_and(image, segm1)
    union = np.logical_or(image, segm1)
    iou = np.sum(intersection) / np.sum(union)
    iou_score.append(iou)
threshold = iou_score.index(np.max(iou_score))
IoU = np.max(iou_score)
th , image_sw = cv2.threshold(gmic_sw, threshold, 255, cv2.THRESH_BINARY) 
inter_out = np.sum(np.logical_and(image_sw, segm1))
if inter_out != 0: 
    labelled, nr_objects = mh.label(image_sw)
    new_image = np.zeros(np.shape(image_sw))
    for i in range(1, nr_objects + 1, 1):
        mask = (labelled == i)
        inter = np.sum(np.logical_and(mask, segm1))
        if inter != 0:
            new_image = np.logical_or(new_image, mask)
    new_image = np.uint8(255*new_image)
    new_image = imageio.core.util.Array(new_image)
    cv2.imwrite(('ImageSweep/15ca_20_R_CC_CAM1SW.png'), new_image)
else:
    gmic_iou_n_.loc[n_] = (res1.split(".")[0], otsu_threshold)
    n_ += 1